# Демонстрация Spell Checker

В этом ноутбуке будет произведена демонстрация написанной модели на примере нескольких предложений.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.models.spelling_correction.levenshtein import (
    LevenshteinSearcherComponent
)
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer, MosesPunctNormalizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

### Tokenizer/Detokenizer

Этот компонент отвечает за токенизаци/детокенизацию исходного предложения. В качестве основы было решено взять токенизатор из библиотеки [sacremoses](https://github.com/alvations/sacremoses).

Опция `escape=False` установлена для корректной работы удаления пунктуации, иначе, например, пунктуационный символ `"` заменяется на `&quot;` и его не удается отследить.

In [5]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

### Candidate Generator

Этот компонент отвечает за то, чтобы генерировать кандидатов для каждой позиции токенизированного предложения. 

На данный момент берутся слова из словаря на заданном расстоянии Дамерау-Левенштейна от исходного токена. Иногда эти слова еще разбиваются пробелами.

В качестве словаря был взят [этот](https://github.com/danakt/russian-words/).

In [6]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
levenshtein_searcher_component = LevenshteinSearcherComponent(
    words=vocab.keys(), max_distance=1
)
candidate_generator = LevenshteinCandidateGenerator(
    levenshtein_searcher_component
)

2021-01-21 23:14:24.624 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/mrgeekman/Documents/MIPT/НИР/Repo/data/external/russian_words/russian_words_vocab.dict]


### Position Selector

Этот компонент отвечает за нахождение оптимальной позиции для замены и предварительный выбор кандидатов для этой позиции.

На данный момент берутся две языковые модели: слева-направо и справа-налево, которые работают с текстом без пунктуации. Для каждой позиции по просматривается весь список кандидатов при использовании левого и правого контекстов и подсчитывается log-prob. Если токен состоит из нескольких подтокенов (например, если в слове есть пробел), то скор суммируется.

Так как для каждого кандидата имеется два log-prob их результат аггрегируется. На данный момент от них считается среднее гармоническое. 

Мы также знаем какой из кандидатов в рассматриваемой позиции соответствует текущему токену, что позволяет нам найти разницу между его скором и скором другого кандидата. Так как мы оперируем с log-prob, то это можно понимать, как логарифм того, насколько изменится вероятность при рассмотрении нового токена вместо изначального.

Выбранная позиция для изменения -- та, на которой достигается наибольшая разница скоров между текущим токеном и одним из кандидатов. На основе разницы скоров так же отбирается предварительный список кандидатов.

In [7]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
position_selector = KenlmPositionSelector(model_left_right, model_right_left)

### Stopping Criteria

Этот компонент отвечает за остановку итераций исправления.

На данный момент рассматривается разница между скорами текущего токена и предварительно лучшего кандидата в выбранной позиции и сравнивается с константой $\alpha$. Так как мы в position selector работали с log-prob, то это можно интерпретировать, как увеличение вероятности в $\exp{\alpha}$ раз. Если это увеличение не превышает заданной константы, то работа алгоритма завершается.

In [8]:
margin_constant = np.log(2)
stopping_criteria = MarginStoppingCriteria(margin_constant)

### Candidate Scorer

Этот компонент отвечает за выбор наилучшего кандидата из списка предложенных position selector.

На данный момент берется модель [Conversational RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html). В первую очередь токенизируется (WordPiece) исходное предложение с MASK-токеном на месте замяемого токена. Хочется просто запустить Masked Language Modeling и попробовать поподставлять кандидатов вместо MASK, но проблема в том, что некоторые кандидаты состоят из более, чем одного токена. В таком случае мы токенизируем всех кандидатов и пытаемся двигать MASK-токен по каждой позиции внутри него, делая другие позиции UNK-токеном и отключая для них attention (не обязательно ставить UNK-токен, это было сделано для удобства). Для аггрегации log-prob скоров внутри одного кандидата берется среднее.

В результате отбирается кандидат с наилучшим скором.

In [9]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
scorer_basis = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = np.mean
candidate_scorer = BertCandidateScorer(scorer_basis, agg_subtoken_func)

Some weights of the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a d

### Spell Checker

Выше были описаны все компоненты модели. Итого, имеем этапы:
1. Генерация кандидатов
2. Итерации до тех пор, пока не сработает критерий останова или не исчерпается максимальное количество итераций
    * Поиск лучшей позиции для исправления и отбор кандидатов для исправления
    * Проверка критерия останова
    * Выбор лучшего исправления
    * Исправление текущего предложения
    * Возможное вынесение некоторых позиций из рассмотрения position selector на следующей итерации.
    
В последнем пункте упомянут механизм вынесения некоторых позиций из рассмотрения. Опишем его более подробно. Дело в том, что иногда position selector находит много мест для возможных исправлений, но в итоге выбирает не самое удачное, на котором candidate scorer выбирает тот токен, который уже там стоит. Это значит, что на следующей итерации ничего не изменится и итерация в точности повторится. Чтобы такого не происходило такие неудачные позиции перестают рассматриваться в position selector до тех пор, пока хоть какая-то коррекция не будет произведена.

In [10]:
# количество кандидатов, отбираемых position selector
num_selected_candidates = 16
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    stopping_criteria,
    tokenizer,
    detokenizer,
    num_selected_candidates,
    max_it
)

## Демонстрация

Возьмем 20 случайных предложений из обучающей части датасета вместе с ответами и посмотрим на результаты.

In [11]:
num_examples = 20

with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    all_sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    all_corrected_sentences = inf.readlines()

In [12]:
np.random.seed(17)
num_sentences = len(all_sentences)
all_indices = np.arange(num_sentences)
np.random.shuffle(all_indices)
indices = all_indices[:num_examples]

examples = [all_sentences[idx].strip() for idx in indices]
examples_true = [all_corrected_sentences[idx].strip() for idx in indices]

Запустим наш spell checker.

In [13]:
examples_corrected = spellchecker(examples)

In [14]:
for i in range(num_examples):
    print(i+1)
    print(f'Original:\t{examples[i]}')
    print(f'Corrected:\t{examples_corrected[i]}')
    print(f'True:\t\t{examples_true[i]}')
    print()

1
Original:	люблю ужастики, но не смотрю теперь итак темноты боюсь, а если посмотрю потом ваще спать не могу )
Corrected:	люблю ужастики но не смотрю теперь и так темноты боюсь а если посмотрю потом ваще спать не могу
True:		люблю ужастики но не смотрю теперь и так темноты боюсь а если посмотрю потом вообще спать не могу

2
Original:	Этот телевизионный коктейль смешан таким образом что с утра нам вдалбливают о проблемах со здоровьем, потом о проблемах на планете, а тут еще и свои запары вдобавок.
Corrected:	этот телевизионный коктейль смешан таким образом что с утра нам вдалбливают о проблемах со здоровьем потом о проблемах на планете а тут еще и свои запасы вдобавок
True:		Этот телевизионный коктейль смешан таким образом что с утра нам вдалбливают о проблемах со здоровьем потом о проблемах на планете а тут еще и свои запары вдобавок

3
Original:	" Ты не видела, где мои носки?
Corrected:	ты не видела где мои носки
True:		Ты не видела где мои носки

4
Original:	Но став старше, она осозн

Разберем каждый пример по-отдельности и проанализируем ошибки:
1. $\pm$ 
    * Корректно исправлено "и так" на "итак".
    * Не исправлено "ваще" на "вообще". Большая разница в расстоянии Дамера-Левенштейна.
2. $+$
    * Нет ошибок, нет исправлений.
3. $+$ 
    * Нет ошибок, нет исправлений.
4. $-$
    * Без нужды исправлено "не" на "ее".
5. $-$
    * Не исправлено "пользоваццо" на "пользоваться". Большая разница в расстоянии Дамерау-Левенштейна.
6. $+$
    * Нет ошибок, нет исправлений.
7. $\pm$
    * Корректно исправлено "некотрые" на "некоторые".
    * Корректно исправлено "ничево" на "ничего".
    * Не исправлено "време ни" на "времени". На данный момент нет механизма по объединению слов, разбитых по пробелу, поэтому, возможно, произошла такая странная аномалия.
    * Без нужды исправлено "доиграть" на "дои рать".
8. $-$
    * Не исправлено "отвественный" на "ответственный". Ошибка произошла в candidate scorer. Кандидат "отвественный" состоял из четырех WordPiece-токенов, а "ответственный" из двух. Так оказалось, что средний скор токенов второго оказался меньше, чем средний скор токенов первого. Если бы аггрегуриющая функция по сабтокенам была суммой, то победил бы кандидат "ответственный".
9. $+$
    * Нет ошибок, нет исправлений.
    * Я не нашел информацию про то есть ли город Магнитогоск, поэтому, возможно, ошибка в оригинальном датасете.
10. $+$
    * Нет ошибок, нет исправлений.
11. $+$
    * Нет ошибок, нет исправлений.
12. $+$
    * Корректно исправлено "хороше" на "хорошо"
13. $+$
    * Нет ошибок, нет исправлений.
14. $-$
    * Не исправлено "ооочень" на "очень". Большая разница в расстоянии Дамерау-Левенштейна.
15. $+$
    * Корректно исправлено "вобщем" на "в общем". 
    * Корректно исправлено "о" на "об".
    * Корректно исправлено "особеностях" на "особенностях".
16. $\pm$
    * Без нужды исправлено "ослуживание" на "обслуживание". (Возможно, ошибка в оригинальном датасете).
    * Корректно исправлено "безупречноне" на "безупречное".
17. $+$
    * Нет ошибок, нет исправлений.
18. $+$
    * Корректно исправлено "Позаввчера" на "позавчера".
19. $\pm$
    * Корректно исправлено "Мужчинаи" на "мужчина и"
    * Некорректно исправлено "идруг" на "друг" вместо "и друг". В списке кандидатов есть "и друг", но каждый его сабтокен получил меньший скор, чем "друг".
20. $+$
    * Нет ошибок, нет исправлений.

## Выводы

1. Продемонстрирована работа модели.
2. Обнаружено, что модель иногда исправляет там, где исправлять не нужно. Возможно, этот эффект можно уменьшить если лучше настроить stopping criteria и взять качественную языковую модель.
3. Обнаружены возможные ошибки в датасете:
    * В предложении 686 про "Магнитогоск".
    * В предложении 368 про "ослуживание".
4. Обнаружены потенциальные ошибки/классы ошибок:
    * При окончании на "ццо" вместо "тся"/"ться".
    * При растягивании гласной, например, "ооочень" вместо "очень".
    * В статье "Automatic spelling correction for russian social media texts" частые паттерны, не улавливаемые при помощи модели на основе расстояния Дамерау-Левенштейна была захардкожены. Их можно вручную внедрять в список кандидатов. Например:
      * ваще -- вообще
      * грит -- говорит
      * щас -- сейчас
5. Требуется поэкспериментировать над:
    * Аггрегирующей функцией по скорам над WordPiece-токенами. Если брать среднее, то появляются ошибки, как в предложении 8, если брать среднее, то будут ошибки при разбиении слова по пробелам, например, в предложении 1 не исправится "итак" на "и так", а в предложении 15 "вобщем" на "в общем". Может быть, надо брать сумму по сабтокенам одного слова и среднее по разным словам.
    * Максимальным обрабатываемым расстоянием Дамера-Левенштейна;
    * Частями position selector, например, использовать более совершенную языковую модель.
6. Следует подумать над механизмом обработки ошибки, когда требуется объединить два токена.